<a href="https://colab.research.google.com/github/AntoinettedeLima/IRP_FYP_20210522/blob/main/Fog_Data_collection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
// Define area of interest (update coordinates as necessary)
var geometry = ee.Geometry.Rectangle([-113.5, 44.5, -108.5, 49.5]);

// Load Sentinel-2 dataset
var sentinel2 = ee.ImageCollection('COPERNICUS/S2')
                  .filterBounds(geometry)
                  .filterDate('2015-06-01', '2024-12-31')
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90))
                  .select(['B4', 'B3', 'B8']);  // Red, Green, NIR

// Load Landsat 8 Collection 2 Tier 1 dataset
var landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                .filterBounds(geometry)
                .filterDate('2013-04-11', '2024-12-31')
                .filter(ee.Filter.lt('CLOUD_COVER', 90))
                .select(['SR_B4', 'SR_B3', 'SR_B5']);  // Red, Green, NIR equivalent for Landsat 8

// Merge the collections
var combinedCollection = sentinel2.merge(landsat8);

// Define the maximum number of images to export
var maxExports = 1500;
var exportCount = 0;

// Retrieve collection size
var collectionSize = combinedCollection.size().getInfo();
print('Combined Collection Size:', collectionSize);

// Adjust start index to continue exporting from a specific point if needed
var startIndex = 820;

if (collectionSize > 0) {
  // Retrieve collection as a list to access each image individually in smaller batches
  var imageList = combinedCollection.toList(collectionSize);

  // Loop through each image in the list and export in small batches, stopping at maxExports
  var batchSize = 5;

  for (var start = startIndex; start < collectionSize && exportCount < maxExports; start += batchSize) {
    for (var i = 0; i < batchSize && (start + i) < collectionSize && exportCount < maxExports; i++) {
      var image = ee.Image(imageList.get(start + i));
      Export.image.toDrive({
        image: image,
        description: 'Exported_Fog_Image_' + (start + i),
        scale: 30,
        region: geometry,
        maxPixels: 1e9
      });
      exportCount++;  // Increment the count of exported images
    }
  }
  print('Total images exported:', exportCount);
} else {
  print("No images found in the specified date range and parameters.");
}
